In [34]:
from mlscorecheck.aggregated import generate_evaluations_with_all_kfolds
import itertools

In [28]:
def multiplicities(configuration):
    mult = {}
    for item in configuration:
        if item[0] not in mult:
            mult[item[0]] = 1
        else:
            mult[item[0]] += 1
    return mult

In [65]:
class IntegerPartitioning:
    def __init__(self, n, m):
        self.n = n
        self.m = m
        self.x = [0]*(m+1)
        self.s = [0]*(m+1)

        for k in range(1, self.m):
            self.x[k] = 1

        self.x[self.m] = self.n - self.m + 1

        for k in range(1, self.m + 1):
            self.s[k] = self.x[k] + self.s[k-1]

    def generate(self):
        while True:
            yield self.x[1:]

            u = self.x[self.m]
            k = self.m
            while k > 0:
                k = k - 1
                if self.x[k] + 2 <= u:
                    break

            if k == 0:
                return

            f = self.x[k] + 1
            s = self.s[k-1]
            while k < self.m:
                self.x[k] = f
                s += f
                self.s[k] = s
                k += 1

            self.x[self.m] = self.n - self.s[self.m-1]


In [66]:
for x in IntegerPartitioning(7, 5).generate():
    print(x)

[1, 1, 1, 1, 3]
[1, 1, 1, 2, 2]


In [136]:
class FoldPartitioning:
    def __init__(self):
        pass

    def generate(self, p, n, n_folds):
        max_items = (p + n) // n_folds
        remainder = (p + n) % n_folds

        ipart = IntegerPartitioning(p, n_folds)

        for ps in ipart.generate():
            if (sum(item > max_items for item in ps) != 0
                    or sum(item == max_items for item in ps) > remainder):
                continue
            #print(ps)

            n_ordinary = len(ps) - sum(item == max_items for item in ps)
            ns = [max_items - p_val + (idx >= n_ordinary) for idx, p_val in enumerate(ps)]

            #print('ns', ns)
            #print('n_ordinary', n_ordinary)

            # distributing the remainders between 0:idx
            #print('ps', ps[:n_ordinary])
            #print(list(itertools.combinations(ps[:n_ordinary], remainder - (len(ps) - n_ordinary))))
            combinations = {tuple(x) for x in itertools.combinations(ps[:n_ordinary],
                                                                    remainder - (len(ps) - n_ordinary))}

            n_variants = []
            for comb in combinations:
                #print('c', comb)
                tmp = copy.copy(ns)
                cdx = len(comb)-1
                pdx = n_ordinary-1
                while cdx >= 0 and pdx >= 0:
                    if comb[cdx] == ps[pdx]:
                        tmp[pdx] += 1

                    if comb[cdx] <= ps[pdx]:
                        pdx -= 1
                    if ps[pdx] <= comb[cdx]:
                        cdx -= 1

                n_variants.append(tmp)

            for ns in n_variants:
                yield ps, ns


In [137]:
for part in FoldPartitioning().generate(5, 8, 3):
    print(part)

([1, 1, 3], [3, 3, 1])
([1, 1, 3], [3, 3, 2])
([1, 2, 2], [3, 2, 2])
([1, 2, 2], [3, 2, 3])


In [ ]:
for part in FoldPartitioning().generate(5, 8, 3):
    print(part)

In [51]:
class MPartition:
    def __init__(self, n, m, max_items, remainder):
        self.n = n
        self.m = m
        self.max_items = max_items
        self.remainder = remainder
        self.x = [0]*(m+1)
        self.s = [0]*(m+1)

        for k in range(1, self.m):
            self.x[k] = 1

        self.x[self.m] = self.n - self.m + 1

        for k in range(1, self.m + 1):
            self.s[k] = self.x[k] + self.s[k-1]

    def next_(self):
        u = self.x[self.m]
        k = self.m
        while k > 0:
            k = k - 1
            if self.x[k] + 2 <= u:
                break

        if k == 0:
            return False

        f = self.x[k] + 1
        s = self.s[k-1]
        while k < self.m:
            self.x[k] = f
            s += f
            self.s[k] = s
            k += 1

        self.x[self.m] = self.n - self.s[self.m-1]
        return True

    def next(self):
        actual = self.x
        print('actual', actual)
        while not (sum(item > self.max_items for item in actual) == 0
                and sum(item == self.max_items for item in actual) <= self.remainder):
            if not self.next():
                print('skipped')
                return False

        ps = copy.copy(actual[1:])
        ns = [0]*len(ps)
        idx = len(ps) - 1
        remainder = self.remainder
        while ps[idx] == self.max_items and remainder > 0:
            ns[idx] = 1
            remainder -= 1
            idx -= 1

        print('ns', ns)

        # distributing the remainders between 0:idx
        to_pick = ps[:(idx+1)]
        combinations = set(tuple(x) for x in itertools.combinations(to_pick, remainder))
        print('combs', combinations)

        n_variants = []
        for comb in combinations:
            print('c', comb)
            tmp = copy.copy(ns)
            jdx = len(comb)-1
            kdx = len(ps[:(idx+1)])-1
            while jdx >= 0 and kdx >= 0:
                if comb[jdx] == ps[kdx]:
                    tmp[kdx] = 1
                    jdx -= 1
                    kdx -= 1
                elif comb[jdx] < ps[kdx]:
                    kdx -= 1
                elif ps[kdx] < comb[jdx]:
                    jdx -= 1
            n_variants.append(tmp)

        print('n_variants', n_variants)

        for variant in n_variants:
            for idx in range(len(variant)):
                variant[idx] += self.max_items - ps[idx]

        print('ps', ps)
        print('ns', n_variants)
        self.next_()



In [52]:
p = 7
n = 15
n_folds = 5
max_items = (p + n) // n_folds
remainder = (p + n) % n_folds

In [53]:
max_items, remainder

(4, 2)

In [54]:
mp = MPartition(p, n_folds, max_items, remainder)

In [59]:
mp.next()

actual [0, 1, 1, 1, 2, 2]
ns [0, 0, 0, 0, 0]
combs {(1, 1), (1, 2), (2, 2)}
c (1, 1)
c (1, 2)
c (2, 2)
n_variants [[0, 1, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 0, 1, 1]]
ps [1, 1, 1, 2, 2]
ns [[3, 4, 4, 2, 2], [3, 3, 4, 2, 3], [3, 3, 3, 3, 3]]


In [6]:
mp = MPartition(7, 3)
while mp.next():
    print(mp.x)

[0, 1, 2, 4]
[0, 1, 3, 3]
[0, 2, 2, 3]


In [9]:
import copy

In [22]:
def multiplicities(configuration):
    mult = {}
    for item in configuration:
        if item[0] not in mult:
            mult[item[0]] = 1
        else:
            mult[item[0]] += 1
    return mult

In [16]:
def generate_fold_structures(p, n, k):
    items_per_fold = (p + n) // k
    n_remainder = (p + n) % k
    print(items_per_fold, n_remainder)
    mpart = MPartition(p, k)
    partitions = [copy.copy(mpart.x[1:])]
    while mpart.next():
        partitions.append(copy.copy(mpart.x[1:]))
    # filtering
    partitions = [partition for partition in partitions
                    if all(part < items_per_fold for part in partition)]
    # complementing
    partitions = [[(part, items_per_fold - part) for part in partition]
                    for partition in partitions]



    return partitions

In [21]:
tmp = generate_fold_structures(8, 12, 3)

6 2


In [27]:
multiplicities(tmp[2])

{2: 2, 4: 1}